In [328]:
import re
from functools import partial

In [329]:
import pandas as pd

In [330]:
df = pd.read_csv('archive/laptops.csv', encoding='latin-1')
#drop the useless columns

df = df.drop(columns='Unnamed: 0')
# fix a small inssue in data


In [332]:
df.columns

Index(['Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu',
       'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros'],
      dtype='object')

In [335]:
df['Ram'].unique()

array(['8GB', '16GB', '4GB', '2GB', '12GB', '6GB', '32GB', '24GB', '64GB'],
      dtype=object)

In [235]:
df = pd.read_csv('archive/laptops.csv', encoding='latin-1')
#drop the useless columns

df = df.drop(columns='Unnamed: 0')
# fix a small inssue in data


In [236]:
def extract_product(product):
    product_list = product.split()
    product = product_list[0]
    if product_list[1].isalpha() or product_list[1][0].isalpha() :
        product += ' ' + product_list[1]
    return product

def extract_pure_resolution(screen_resolution):
    return re.search('[0-9]{3,4}x[0-9]{3,4}', screen_resolution).group()


In [267]:
df['Convertible'] = df['TypeName'] == '2 in 1 Convertible'
df['Product'] = df['Product'].apply(extract_product)
df['Retina Display'] = df['ScreenResolution'].str.contains('Retina Display')
df['Touchscreen'] = df['ScreenResolution'].str.contains("Touchscreen")
df['IPS Panel'] = df['ScreenResolution'].str.contains('IPS Panel')
df['Quad HD+'] = df['ScreenResolution'].str.contains('Quad HD+')
df['Ultra HD'] = df['ScreenResolution'].str.contains('Ultra HD')
df['ScreenResolution'] = df['ScreenResolution'].apply(extract_pure_resolution)

In [268]:
df['CPU brand'] = df['Cpu'].apply(lambda x: x.split()[0])
df['Cpu speed'] = df['Cpu'].apply(lambda x: re.search('([0-9]{1,2}\.[0-9]{1,3}GHz)|([0-9]GHz)', x)[0])
df['Cpu type'] = df['Cpu'].apply(lambda x: x.split()[1])
df['free OpSys'] = df['OpSys'].apply(lambda x: x in ('Linux', 'Android', 'No OS'))

In [269]:
df[df['Cpu type']=='Pentium']['Cpu']

49      Intel Pentium Quad Core N4200 1.1GHz
166     Intel Pentium Quad Core N4200 1.1GHz
305     Intel Pentium Quad Core N4200 1.1GHz
319     Intel Pentium Quad Core N3710 1.6GHz
365     Intel Pentium Quad Core N3710 1.6GHz
443     Intel Pentium Quad Core N4200 1.1GHz
455     Intel Pentium Quad Core N3710 1.6GHz
465     Intel Pentium Quad Core N4200 1.1GHz
537     Intel Pentium Quad Core N3710 1.6GHz
568     Intel Pentium Quad Core N4200 1.1GHz
571     Intel Pentium Quad Core N3710 1.6GHz
609     Intel Pentium Quad Core N3710 1.6GHz
766     Intel Pentium Quad Core N4200 1.1GHz
769     Intel Pentium Quad Core N4200 1.1GHz
779     Intel Pentium Dual Core N4200 1.1GHz
804     Intel Pentium Quad Core N4200 1.1GHz
825     Intel Pentium Dual Core 4405U 2.1GHz
856     Intel Pentium Quad Core N4200 1.1GHz
876     Intel Pentium Quad Core N4200 1.1GHz
884     Intel Pentium Quad Core N3710 1.6GHz
930     Intel Pentium Quad Core N4200 1.1GHz
941     Intel Pentium Quad Core N3710 1.6GHz
957     In

In [270]:
df['Memory'] = df['Memory'].apply(lambda x: x.replace('.0', ''))

def process_memory(memory, _type):
    gb_match = re.match('([0-9]{1,4})GB ' + _type, memory)
    if gb_match:
        return int(gb_match.groups()[0])
    
    tb_match = re.match('([0-9]{1,4})TB ' + _type, memory)

    if tb_match:
        return int(tb_match.groups()[0]) * 1000

    return 0


def process_combined_memory(_type, memory):
    split_memory = memory.split(' +  ')
    res = sum(process_memory(mem, _type) for mem in split_memory)
    return res

for memory_type in types:
    df[memory_type] = df['Memory'].apply(partial(process_combined_memory, memory_type))


In [271]:
df

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,...,Quad HD+,Ultra HD,CPU brand,Cpu speed,Cpu type,free OpSys,SSD,HDD,Flash Storage,Hybrid
0,Apple,MacBook Pro,Ultrabook,13.3,2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,...,False,False,Intel,2.3GHz,Core,False,128,0,0,0
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,...,False,False,Intel,1.8GHz,Core,False,0,0,128,0
2,HP,250 G6,Notebook,15.6,1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,...,False,False,Intel,2.5GHz,Core,True,256,0,0,0
3,Apple,MacBook Pro,Ultrabook,15.4,2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,...,False,False,Intel,2.7GHz,Core,False,512,0,0,0
4,Apple,MacBook Pro,Ultrabook,13.3,2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,...,False,False,Intel,3.1GHz,Core,False,256,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,Yoga,2 in 1 Convertible,14.0,1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,...,False,False,Intel,2.5GHz,Core,False,128,0,0,0
1299,Lenovo,Yoga,2 in 1 Convertible,13.3,3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,...,True,False,Intel,2.5GHz,Core,False,512,0,0,0
1300,Lenovo,IdeaPad,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,...,False,False,Intel,1.6GHz,Celeron,False,0,0,64,0
1301,HP,15-AC110nv,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,...,False,False,Intel,2.5GHz,Core,False,0,1000,0,0


In [247]:
df['GPU_provider'] = df['Gpu'].apply(lambda x: x.split()[0])

In [276]:
import math

def resolution_to_number(resolution, diagonal):
    width, height = map(int, resolution.split('x'))
    return math.sqrt(width**2+height**2)/diagonal

In [279]:
df.columns

Index(['Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu',
       'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros', 'Convertible',
       'Retina Display', 'Touchscreen', 'IPS Panel', 'Quad HD+', 'Ultra HD',
       'CPU brand', 'Cpu speed', 'Cpu type', 'free OpSys', 'SSD', 'HDD',
       'Flash Storage', 'Hybrid'],
      dtype='object')

In [280]:
df['Pixels Per Inch'] = df.apply(lambda x: resolution_to_number(x.ScreenResolution, x.Inches), axis=1)